In [109]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [2]:
train_df=pd.read_csv('/Users/maksimtrebusinin/Downloads/titanic/train.csv',index_col=0)
test_df=pd.read_csv('/Users/maksimtrebusinin/Downloads/titanic/test.csv',index_col=0)
pred_df=pd.read_csv('/Users/maksimtrebusinin/Downloads/titanic/gender_submission.csv')

In [4]:
train_df.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [20]:
train_df['Age'].unique()

array([22.  , 38.  , 26.  , 35.  ,   nan, 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  , 28.  ,
        8.  , 19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  ,
       49.  , 29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  ,
       16.  , 25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  ,
       71.  , 37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 ,
       51.  , 55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  ,
       45.5 , 20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  ,
       60.  , 10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  ,
       70.  , 24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  ])

In [77]:
regression_for_age_data_test=train_df[(train_df['Age'].isna())==True].drop(['Age','Cabin'], axis=1)

In [78]:
regression_for_age_data_train=train_df[(train_df['Age'].isna())==False].drop('Cabin', axis=1)

In [71]:
test_df.isna().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [80]:
regression_for_age_data_train.isna().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    2
dtype: int64

In [84]:
regression_for_age_data_train.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked     object
dtype: object

In [85]:
le=LabelEncoder()
for col in regression_for_age_data_train.columns:
    if(regression_for_age_data_train[col].dtype=='object'):
        regression_for_age_data_train[col]=le.fit_transform(regression_for_age_data_train[col])

In [89]:
regression_for_age_data_train['Embarked']=regression_for_age_data_train['Embarked'].fillna(2)

In [93]:
for col in regression_for_age_data_test.columns:
    if(regression_for_age_data_test[col].dtype=='object'):
        regression_for_age_data_test[col]=le.fit_transform(regression_for_age_data_test[col])

In [111]:
X_regression_for_age_data_train=regression_for_age_data_train.drop('Age',axis=1)
y_regression_for_age_data_train=regression_for_age_data_train['Age']

In [112]:
regressor=RandomForestRegressor()
parametres={'n_estimators': range(2,16,2), 'max_depth': range(1,14), 'min_samples_split': [2,3,4,5,6,7,8]}
grid_reg=GridSearchCV(regressor, parametres, cv=6, n_jobs=-1)
grid_reg.fit(X_regression_for_age_data_train,y_regression_for_age_data_train)
print(grid_reg.best_params_)

{'max_depth': 5, 'min_samples_split': 8, 'n_estimators': 14}


In [113]:
Age=grid_reg.best_estimator_.predict(regression_for_age_data_test)

In [115]:
regression_for_age_data_test['Age']=Age

In [117]:
regression_for_age_data_test.describe()

,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,Age
count,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000
mean,0.293785,2.598870,88.000000,0.700565,0.564972,0.180791,79.276836,22.158567,1.293785,29.549138
std,0.456787,0.763216,51.239633,0.459310,1.626316,0.534145,45.120510,31.874608,0.800295,7.470978
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.465934
25%,0.000000,3.000000,44.000000,0.000000,0.000000,0.000000,40.000000,7.750000,1.000000,27.708438
50%,0.000000,3.000000,88.000000,1.000000,0.000000,0.000000,82.000000,8.050000,2.000000,30.466741
75%,1.000000,3.000000,132.000000,1.000000,0.000000,0.000000,120.000000,24.150000,2.000000,31.922842
max,1.000000,3.000000,176.000000,1.000000,8.000000,2.000000,154.000000,227.525000,2.000000,47.787636


In [124]:
regression_for_age_data_test=regression_for_age_data_test[['Survived','Pclass','Name','Sex','Age','SibSp',
                                                          'Parch','Ticket','Fare','Embarked']]

In [156]:
result=pd.concat([regression_for_age_data_train,regression_for_age_data_test])

In [157]:
final_train_data=result.sort_index()

In [159]:
final_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    int64  
 3   Sex       891 non-null    int64  
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    int64  
 8   Fare      891 non-null    float64
 9   Embarked  891 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 76.6 KB


In [234]:
serv=final_train_data

In [237]:
final_train_data=final_train_data.drop('Survived',axis=1)
final_train_data['Survived']=serv['Survived']

In [238]:
final_train_data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived
PassengerId,,,,,,,,,,
1,3,103,1,22.000000,1,0,408,7.2500,2,0
2,1,179,0,38.000000,1,0,471,71.2833,0,1
3,3,317,0,26.000000,0,0,532,7.9250,2,1
4,1,247,0,35.000000,1,0,40,53.1000,2,1
5,3,15,1,35.000000,0,0,373,8.0500,2,0
...,...,...,...,...,...,...,...,...,...,...
887,2,461,1,27.000000,0,0,83,13.0000,2,0
888,1,274,0,19.000000,0,0,12,30.0000,2,1
889,3,43,0,27.868321,1,2,153,23.4500,2,0


In [170]:
test_df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [172]:
Survived=pred_df['Survived']

In [194]:
test_df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [195]:
new_df=test_df.merge(pred_df, on='PassengerId')

In [199]:
test_df=new_df.set_index('PassengerId')

In [200]:
test_df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0


In [201]:
regression_for_age_data_test_test=test_df[(test_df['Age'].isna())==True].drop(['Age','Cabin'], axis=1)

In [204]:
for col in regression_for_age_data_test_test.columns:
    if(regression_for_age_data_test_test[col].dtype=='object'):
        regression_for_age_data_test_test[col]=le.fit_transform(regression_for_age_data_test_test[col])

In [205]:
regression_for_age_data_test_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 902 to 1309
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    86 non-null     int64  
 1   Name      86 non-null     int64  
 2   Sex       86 non-null     int64  
 3   SibSp     86 non-null     int64  
 4   Parch     86 non-null     int64  
 5   Ticket    86 non-null     int64  
 6   Fare      86 non-null     float64
 7   Embarked  86 non-null     int64  
 8   Survived  86 non-null     int64  
dtypes: float64(1), int64(8)
memory usage: 6.7 KB


In [208]:
regression_for_age_data_test_test=regression_for_age_data_test_test[['Survived','Pclass','Name','Sex','SibSp',
                                                          'Parch','Ticket','Fare','Embarked']]

In [209]:
regression_for_age_data_test_test

,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
902,0,3,21,1,0,0,34,7.8958,2
914,1,1,9,0,0,0,73,31.6833,2
921,0,3,70,1,2,0,16,21.6792,0
925,1,3,23,0,1,2,80,23.4500,2
928,1,3,62,0,0,0,30,8.0500,2
...,...,...,...,...,...,...,...,...,...
1300,1,3,60,0,0,0,27,7.7208,1
1302,1,3,50,0,0,0,44,7.7500,1
1305,0,3,75,1,0,0,63,8.0500,2


In [210]:
Age1=grid_reg.best_estimator_.predict(regression_for_age_data_test_test)

In [211]:
regression_for_age_data_test_test['Age']=Age1

/var/folders/8d/5216m_0563lf5wxz_91d1k980000gn/T/ipykernel_1803/4155913647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regression_for_age_data_test_test['Age']=Age1


In [212]:
regression_for_age_data_test_test

,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,Age
PassengerId,,,,,,,,,,
902,0,3,21,1,0,0,34,7.8958,2,29.505275
914,1,1,9,0,0,0,73,31.6833,2,40.342226
921,0,3,70,1,2,0,16,21.6792,0,30.989278
925,1,3,23,0,1,2,80,23.4500,2,20.667456
928,1,3,62,0,0,0,30,8.0500,2,27.927497
...,...,...,...,...,...,...,...,...,...,...
1300,1,3,60,0,0,0,27,7.7208,1,25.101645
1302,1,3,50,0,0,0,44,7.7500,1,25.101645
1305,0,3,75,1,0,0,63,8.0500,2,30.891518


In [213]:
regression_for_age_data_test_test=regression_for_age_data_test_test[['Survived','Pclass','Name','Sex','Age','SibSp',
                                                          'Parch','Ticket','Fare','Embarked']]

In [216]:
regression_for_age_data_test_test=regression_for_age_data_test_test.drop('Survived',axis=1)
test_df=test_df.drop('Survived',axis=1)

In [219]:
test_df=test_df.drop('Cabin',axis=1)
for col in test_df.columns:
    if(test_df[col].dtype=='object'):
        test_df[col]=le.fit_transform(test_df[col])

In [222]:
test_df=test_df.dropna()

In [227]:
final_test_data=pd.concat([test_df,regression_for_age_data_test_test]).sort_index()

In [230]:
final_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    int64  
 2   Sex       418 non-null    int64  
 3   Age       418 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    int64  
 7   Fare      418 non-null    float64
 8   Embarked  418 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 32.7 KB


In [240]:
final_train_data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived
PassengerId,,,,,,,,,,
1,3,103,1,22.000000,1,0,408,7.2500,2,0
2,1,179,0,38.000000,1,0,471,71.2833,0,1
3,3,317,0,26.000000,0,0,532,7.9250,2,1
4,1,247,0,35.000000,1,0,40,53.1000,2,1
5,3,15,1,35.000000,0,0,373,8.0500,2,0
...,...,...,...,...,...,...,...,...,...,...
887,2,461,1,27.000000,0,0,83,13.0000,2,0
888,1,274,0,19.000000,0,0,12,30.0000,2,1
889,3,43,0,27.868321,1,2,153,23.4500,2,0


In [243]:
st=StandardScaler()
st.fit(X_train)

StandardScaler()

In [250]:
y=pred_df['Survived']

In [245]:
X_train_trans=pd.DataFrame(st.transform(X_train), columns=X_train.columns)
X_test_trans=pd.DataFrame(st.transform(final_test_data), columns=final_test_data.columns)

In [276]:
clf_svc=SVC()
parametres_svc={'C': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]}
grid_svc=GridSearchCV(clf_svc, parametres_svc, cv=7)
grid_svc.fit(X_train_trans,y_train)
print(grid_svc.best_params_)

{'C': 0.8}


In [277]:
y_pred_svc=grid_svc.best_estimator_.predict(X_test_trans)
print(accuracy_score(y,y_pred_svc))
print(precision_score(y,y_pred_svc))
print(recall_score(y,y_pred_svc))
print(f1_score(y,y_pred_svc))

0.9401913875598086
0.9096774193548387
0.9276315789473685
0.9185667752442996


In [280]:
new_output=pd.DataFrame({'PassengerId': pred_df.PassengerId, 'Survived': y_pred_svc})
new_output.to_csv('submission_5.csv', index=False)